In [ ]:
from matplotlib.pyplot import *
import numpy as np
import pandas as pd
import os
import importlib


from ATARI.sammy_interface import sammy_interface, sammy_classes, sammy_functions, template_creator

from ATARI.ModelData.particle_pair import Particle_Pair

from ATARI.ModelData.experimental_model import Experimental_Model

from copy import copy

from ATARI.AutoFit import chi2_eliminator_v2
from ATARI.AutoFit import elim_addit_funcs


In [ ]:
%matplotlib widget

## Measurement Data

The following measurement data are imported from the work of Jesse Brown at RPI (currently at ORNL).
These measurements are detailed in https://www.tandfonline.com/doi/full/10.1080/00295639.2023.2249786 and consist of 6 experimental configurations.
Of the 4 transmission measurements only 3 are used for evaluation and the 4th (45m FP) is used for validation. 
Both 2 capture measurements are used in evaluation.

The transmission data are reliable in the range 0.15-100 keV.
The capture data are reliable from 0.15-200 keV. 



In [ ]:
### Determine channel widths
from ATARI.theory.experimental import e_to_t, t_to_e

def get_chw_and_upperE(E, FP):
    E = np.array(E)
    tof = e_to_t(E, FP, True)
    dt = np.diff(tof*1e6)
    widths1, index1 = np.unique(np.round(dt, 4), return_index=True)
    chw, Emax = np.flipud(widths1), np.flipud(E[index1])
    strc = ''
    stre = ''
    for c,e in zip(chw, Emax):
        strc += f"{c*1e3:.2f}, "
        stre += f"{e:.2f}, "
    return stre, strc

# Emax, chw = get_chw_and_upperE(transdat6.E, 100.14)
# Emax, chw = get_chw_and_upperE(capdat1.E, 45.27)
# print(Emax)
# print(chw)

energy_range_all = [202, 227]
# energy_range_all = [201, 228]

In [ ]:
### 1mm capture data
capdat1 = sammy_functions.readlst("./yield_ta1b_unsmooth.dat")
expcap1 = Experimental_Model(title = "cap1mm",
                                reaction ="capture", 
                                energy_range = energy_range_all,
                                n = (0.005631, 0),
                                FP = (45.27, 0.05),
                                burst= (8.0,1.0), 
                                temp= (294.2610, 0.0),
                                channel_widths={
                                    "maxE": [68.20,  122.68, 330.48, 547.57, 199359.52], 
                                    "chw":  [821.40, 410.70, 102.70, 51.30,  25.70],
                                    "dchw": [0.8,    0.8,    0.8,    0.8,    0.8]
                                }
                               )
capdat1 = capdat1.loc[(capdat1.E<max(expcap1.energy_range)) & (capdat1.E>min(expcap1.energy_range)), :]


### 2mm capture data
capdat2 = sammy_functions.readlst("./yield_ta2_unsmooth.dat")
expcap2 = Experimental_Model(   title = "cap2mm",
                                reaction = "capture", 
                                energy_range = energy_range_all,
                                n = (0.011179, 0.0),
                                FP = (45.27, 0.05),
                                burst = (8.0,1.0),
                                temp = (294.2610, 0.0),
                                channel_widths={
                                    "maxE": [68.20,  122.68, 330.48, 547.57, 199359.52], 
                                    "chw":  [821.40, 410.70, 102.70, 51.30,  25.70],
                                    "dchw": [0.8,    0.8,    0.8,    0.8,    0.8]
                                }
                               )
capdat2 = capdat2.loc[(capdat2.E<max(expcap2.energy_range)) & (capdat2.E>min(expcap2.energy_range)), :]


### 1mm Transmission data
transdat1 = sammy_functions.readlst("./trans-Ta-1mm.twenty")
transdat1_covfile = './trans-Ta-1mm.idc'
chw, Emax = get_chw_and_upperE(transdat1.E, 100.14)
exptrans1 = Experimental_Model(title = "trans1mm",
                               reaction = "transmission", 
                               energy_range = energy_range_all,

                                n = (0.00566,0.0),  
                                FP = (100.14,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat1 = transdat1.loc[(transdat1.E<max(exptrans1.energy_range)) & (transdat1.E>min(exptrans1.energy_range)), :]

### 3mm transmission data
transdat3 = sammy_functions.readlst("./trans-Ta-3mm.twenty")
transdat3_covfile = "./trans-Ta-3mm.idc"
exptrans3 = Experimental_Model(title = "trans3mm",
                               reaction = "transmission", 
                               energy_range = energy_range_all,

                                n = (0.017131,0.0),  
                                FP = (100.14,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat3 = transdat3.loc[(transdat3.E<max(exptrans3.energy_range)) & (transdat3.E>min(exptrans3.energy_range)), :]


### 6mm transmission data
transdat6 = sammy_functions.readlst("./trans-Ta-6mm.twenty")
transdat6_covfile = "./trans-Ta-6mm.idc"
exptrans6 = Experimental_Model(title = "trans6mm",
                               reaction = "transmission", 
                               energy_range = energy_range_all,

                                n = (0.03356,0.0),  
                                FP = (100.14,0.0), 
                                burst = (8, 0.0), 
                                temp = (294.2610, 0.0),

                               channel_widths={
                                    "maxE": [216.16, 613.02, 6140.23], 
                                    "chw": [204.7, 102.4, 51.2],
                                    "dchw": [1.6, 1.6, 1.6]
                                }
                                
                               )
transdat6 = transdat6.loc[(transdat6.E<max(exptrans6.energy_range)) & (transdat6.E>min(exptrans6.energy_range)), :]


### Not using 12mm measurement for evaluation - this is a validation measurement

# transdat12 = sammy_functions.readlst("/Users/noahwalton/research_local/resonance_fitting/ATARI_workspace/measurement_data/trans-Ta-12mm.dat")
# # transdat12_covfile = Need to generate from sys and stat covariances
# exptrans12 = Experimental_Model(title = "trans12",
#                                 reaction = "transmission",
#                                 energy_range = erange_all,

#                                 sammy_inputs = {
#                                     'alphanumeric'       :   ["BROADENING IS WANTED"],
#                                     'ResFunc'            :   "ORRES"
#                                         },

#                                 n = (0.067166, 0.0),  
#                                 FP = (35.185,0.0), 
#                                 burst = (8,0.0), 
#                                 temp = (294.2610, 0.0),

#                                 channel_widths={
#                                         "maxE": [270], 
#                                         "chw": [102.7],
#                                         "dchw": [0.8]
#                                         },

#                                 additional_resfunc_lines=["WATER 0004 5.6822000 -0.54425 0.07733000", "WATER      0.5000000  0.05000 0.00700000", "LITHI 000  -1.000000  -1.0000 6.00000000", "LITHI      0.1000000  0.10000 0.60000000", "LITHI      166.87839 -28.7093 1.260690", "LITHI      0.2574580 -0.06871 0.004915"]
#                                )

# transdat12 = transdat12[(transdat12.E<max(exptrans12.energy_range)) & (transdat12.E>min(exptrans12.energy_range))]


In [ ]:
### plotting function
def plot(datasets, experiments, fits=[], priors=[], true=[]):
    colors = ["C1", "C2", "C3", "C4", "C5", "C6", "C7"]
    fig, axes = subplots(2,1, figsize=(8,6), sharex=True)

    for i, exp in enumerate(experiments):
        if exp.reaction == "transmission":
            model_key = "theo_trans"
            iax = 0
        elif exp.reaction == "capture":
            model_key = "theo_xs"
            iax = 1
        else:
            raise ValueError()

        axes[iax].errorbar(datasets[i].E, datasets[i].exp, yerr=datasets[i].exp_unc, zorder=0,
                                                fmt='.', color=colors[i], linewidth=0.5, markersize=1.5, capsize=1, label=exp.title)
        
        if len(fits) != 0:
            axes[iax].plot(fits[i].E, fits[i][model_key], color=colors[i], lw=1.5, label=f'fit {exp.title}')

        if len(priors) != 0:
            axes[iax].plot(priors[i].E, priors[i][model_key], '--', color=colors[i], lw=1.5) #, label=f'prior {exp.title}')
        if len(true) != 0:
            axes[iax].plot(true[i].E, true[i][model_key], '-', color=colors[i], alpha=0.5, lw=1.5) #, label=f'prior {exp.title}')

        
    axes[0].set_ylabel("T")
    axes[1].set_ylabel(r"$Y_{\gamma}$")

    ### make it pretty
    for ax in axes:

        ax.set_xscale('log')
        ax.set_ylim([-0.1,1.1])
        ax.legend()

    fig.supxlabel('Energy (eV)')
    fig.tight_layout()

    return fig



In [ ]:
### setup in zipped lists 
datasets = [capdat1, capdat2, transdat1, transdat3, transdat6]
experiments= [expcap1, expcap2, exptrans1, exptrans3, exptrans6]
covariance_data = [{}, {}, transdat1_covfile, transdat3_covfile, transdat6_covfile]
templates = []
for data, exp in zip(datasets, experiments):
    filepath = f'template_{exp.title}_edited'
    exp.template = os.path.realpath(filepath)


fig = plot(datasets, experiments)
fig.tight_layout()

In [ ]:
## Could also plot covariance here

In [ ]:
# all options to run! 
sammypath = '/home/fire/SAMMY/sammy/build/install/bin/sammy'

# folder where to save all
savefolder = './data/'
fig_size = (10,6)

Gn_thr = 0.01
N_red = 100 # number of resonances to keep after the initial autofit

chi2_allowed = 10000

## Fit from ENDF or JEFF

In [ ]:
sammyRTO = sammy_classes.SammyRunTimeOptions(#'/Users/noahwalton/gitlab/sammy/sammy/build/bin/sammy',
                            sammyexe=sammypath,
                            options= {"Print"   :   True,
                              "bayes"   :   False,
                              "keep_runDIR"     : True
                              })

matnum = 7328

# endf_file = "/Users/noahwalton/research_local/data/neutrons_ENDF-B-VII.1/n-073_Ta_181.endf"
# endf_parameters = sammy_functions.get_endf_parameters(endf_file, matnum, sammyRTO)

# endf_parameters = endf_parameters[(endf_parameters.E<260) & (endf_parameters.E>190)]
# endf_parameters["varyGn1"] = np.ones(len(endf_parameters))
# endf_parameters["varyGg"] = np.ones(len(endf_parameters))*0
# endf_parameters["varyE"] = np.ones(len(endf_parameters))

jeff_file = "/Users/noahwalton/research_local/data/JEFF33_endf6/73-Ta-181g.jeff33"
jeff_file = "./73-Ta-181g.jeff33"
jeff_parameters = sammy_functions.get_endf_parameters(jeff_file, matnum, sammyRTO)

jeff_parameters = jeff_parameters[(jeff_parameters.E<max(energy_range_all)+5) & (jeff_parameters.E>min(energy_range_all)-5)]
jeff_parameters["varyGn1"] = np.ones(len(jeff_parameters))
jeff_parameters["varyGg"] = np.ones(len(jeff_parameters))*0
jeff_parameters["varyE"] = np.ones(len(jeff_parameters))

jeff_parameters

In [ ]:

Ta_pair = Particle_Pair(isotope="Ta181",
                        formalism="XCT",
                        ac=8.1271,     # scattering radius
                        M=180.948030,  # amu of target nucleus
                        m=1,           # amu of incident neutron
                        I=3.5,         # intrinsic spin, positive parity
                        i=0.5,         # intrinsic spin, positive parity
                        l_max=2)       # highest order l-wave to consider

Ta_pair.add_spin_group(Jpi='3.0',
                       J_ID=1,
                       D_avg=8.79,
                       Gn_avg=46.5,
                       Gn_dof=1,
                       Gg_avg=64.0,
                       Gg_dof=1000)

Ta_pair.add_spin_group(Jpi='4.0',
                       J_ID=2,
                       D_avg=4.99,
                       Gn_avg=35.5,
                       Gn_dof=1,
                       Gg_avg=64.0,
                       Gg_dof=1000)


rto = sammy_classes.SammyRunTimeOptions(
    sammyexe=sammypath,
                             options = {"Print"   :   True,
                              "bayes"   :   True,
                              "keep_runDIR"     : True
                              })


sammyINPyw = sammy_classes.SammyInputDataYW(
    particle_pair = Ta_pair,
    resonance_ladder = jeff_parameters,  

    datasets= datasets,
    experiments = experiments,
    experimental_covariance=covariance_data,  #[{}, {}, {}, {}, {}], # 
    
    max_steps = 10,
    iterations = 2,
    step_threshold = 0.1,
    autoelim_threshold = None,

    LS = False,
    LevMar = True,
    LevMarV = 2,
    LevMarVd= 5,
    initial_parameter_uncertainty = 0.1
    )

print('Spin group keys:')
print(list(Ta_pair.spin_groups.keys()))

In [ ]:
# size of the dataset - to compare

# dataset size
sum_points = 0 
for index,el in enumerate(datasets):
    sum_points += el.shape[0]

print(f'Datasets: {len(datasets)}')
print(f'Num of points: {sum_points}')

In [ ]:
sammyOUT_SFJ = sammy_functions.run_sammy_YW(sammyINPyw, rto)

In [ ]:
# prior_lsts = lsts
def printout_chi2_post(sammyOUT: sammy_classes.SammyOutputData, addstr :str = 'Solution chi2 values'):
    print(f'{addstr}')
    print('Chi2_n:')
    print('\t Prior:')
    print('\t', sammyOUT.chi2n, np.sum(sammyOUT.chi2n))
    print('\t Posterior:')
    print('\t', sammyOUT.chi2n_post, np.sum(sammyOUT.chi2n_post))

    print('Chi2:')
    print('\t Prior:')
    print('\t', sammyOUT.chi2, np.sum(sammyOUT.chi2))
    print('\t Posterior:')
    print('\t', sammyOUT.chi2_post, np.sum(sammyOUT.chi2_post))

printout_chi2_post(sammyOUT_SFJ, 'JEFF')

In [ ]:
# fig2 = plot(datasets, experiments, fits=sammyOUT.pw_post, priors=sammyOUT.pw)
# fig2.tight_layout()

# plotting using modified func

fig2 = plot_datafits(datasets, experiments, 
    fits = sammyOUT_SFJ.pw_post, 
    fits_chi2 = sammyOUT_SFJ.chi2_post, 
    f_model_name = 'SFJ post',
    
    priors = sammyOUT_SFJ.pw, priors_chi2 = sammyOUT_SFJ.chi2, pr_model_name='SFJ prior',

    #true = sammyOUT_SFJ.pw, 
    #true_chi2 = sammyOUT.chi2, 
    t_model_name ='SFJ prior',
    true_pars = Ta_pair.resonance_ladder,
    
    fit_pars = sammyOUT_SFJ.par_post,
    prior_pars = sammyOUT_SFJ.par,
      
    title = 'Models Comparison',
    show_spingroups = False,
    #fig_size = fig_size
    )

fig2.savefig(fname=savefolder+'SFJ_Fit_Result.png')

In [ ]:
sammyOUT_SFJ.par_post

In [ ]:
# utilizing the autofit from initial FB


from ATARI.AutoFit.initial_FB_solve import InitialFB, InitialFBOPT

sammy_rto_fit = sammy_classes.SammyRunTimeOptions(
    sammyexe=sammypath,
                             options = {"Print"   :   True,
                              "bayes"   :   True,
                              "keep_runDIR"     : True
                              })


options = InitialFBOPT(GGn_threshold=Gn_thr,
                       iterations=3,
                       max_steps = 30,
                       step_threshold=0.01,
                       LevMarV0=0.05,
                       fit_all_spin_groups=False,
                       spin_group_keys = ['3.0']
                       )

autofit_initial = InitialFB(options)

In [ ]:
outs = autofit_initial.fit(Ta_pair,
                               energy_range_all,
                               datasets,
                               experiments,
                               covariance_data,
                               sammy_rto_fit)

In [ ]:
print(f'Num elements in outs (stages): {len(outs)}')
printout_chi2_post(outs[-1], 'autofit result')
#print(outs[-1].par_post)

printout_chi2_post(outs[0], 'autofit prior')

In [ ]:
# saving initial solution & chars
f_name_to_save = f'Autofit_init_res_sf_{N_red}_er[{np.min(energy_range_all)}_{np.max(energy_range_all)}]_chi2allowed_{chi2_allowed}.pkl'

save_obj  = elim_addit_funcs.save_obj_as_pkl(folder_name=savefolder, 
                                             file_name=f_name_to_save,
                                             obj = outs)

In [ ]:
# reading the prefitted data
f_name_to_load = f_name_to_save

outs = elim_addit_funcs.load_obj_from_pkl(folder_name=savefolder, 
                                          pkl_fname=f_name_to_load)
final_fb_output = outs[-1]

In [ ]:
# fig = plot(datasets, experiments, fits=final_fb_output.pw_post, priors=final_fb_output.pw)
# fig.tight_layout()

# plotting to show pos. of resonances
fig2 = elim_addit_funcs.plot_datafits(datasets, experiments, 
    fits = final_fb_output.pw_post, 
    fits_chi2=final_fb_output.chi2_post, 
    f_model_name='AF post',
    
    priors=final_fb_output.pw, priors_chi2=final_fb_output.chi2, pr_model_name='AF prior',

    true=sammyOUT_SFJ.pw, # TODO: philosophy
    true_chi2 = sammyOUT_SFJ.chi2, 
    t_model_name ='JEFF prior',
    true_pars = jeff_parameters,
    fit_pars = final_fb_output.par_post,
    prior_pars = final_fb_output.par,
      
    title = 'Models Comparison',
    show_spingroups = True,
    #fig_size = fig_size
    )

fig2.savefig(fname=savefolder+'Initial_Fit_Result.png')

In [ ]:
print(final_fb_output.chi2)
print(sum(final_fb_output.chi2))
print(final_fb_output.chi2_post)
print(sum(final_fb_output.chi2_post))
print(f'N_res: {final_fb_output.par_post.shape[0]}')

# Resonance elimination 


In [ ]:
import importlib

importlib.reload(chi2_eliminator_v2)
importlib.reload(elim_addit_funcs)

start_ladder = final_fb_output.par_post
assert isinstance(start_ladder, pd.DataFrame)

# side resonances if needed, otherways - keep empty
side_resonances_df = pd.DataFrame()

side_resonances_df = elim_addit_funcs.find_side_res_df(
        initial_sol_df = jeff_parameters,
        energy_region = energy_range_all,
        N_res = 2
)

side_resonances_df = elim_addit_funcs.set_varying_fixed_params(ladder_df=side_resonances_df,
                                                               vary_list=[0,1,1])
print('Side resonances:')
print(side_resonances_df)


# compiling to one ladder
start_ladder = pd.concat([side_resonances_df, start_ladder], ignore_index=True)
print('Final ladder to eliminate from')
print(start_ladder)


In [ ]:
# if we do not want to wait hours...

N_red = min(N_red, start_ladder.shape[0],) # not limiting

# just to reduce processing time
start_ladder = elim_addit_funcs.reduce_ladder(ladder_df=start_ladder,
                             Gn1_threshold=Gn_thr,
                             vary_list=[1,0,1],
                             N=N_red,
                             keep_fixed=True,
                             fixed_side_resonances=side_resonances_df)

print('Start ladder:')
print(start_ladder)


In [ ]:

# defining rto & inputs
elim_sammyINPyw = sammy_classes.SammyInputDataYW(
    particle_pair = Ta_pair,
    resonance_ladder = start_ladder,

    datasets = datasets,
    experimental_covariance=covariance_data,
    experiments = experiments,

    max_steps = 0,
    iterations = 2,
    step_threshold = 0.01,
    autoelim_threshold = None,

    LS = False,
    LevMar = True,
    LevMarV = 1.5,

    minF = 1e-5,
    maxF = 10,
    initial_parameter_uncertainty = 0.05
    )

# defining the elim_opts
elim_opts = chi2_eliminator_v2.elim_OPTs(chi2_allowed = chi2_allowed,
                                         stop_at_chi2_thr = False,
                                      fixed_resonances_df = side_resonances_df,
                                      deep_fit_max_iter = 30,
                                      deep_fit_step_thr = 0.01,
                                      start_fudge_for_deep_stage = 0.05,
                                      )

In [ ]:
elimi = chi2_eliminator_v2.eliminator_by_chi2(rto=sammy_rto_fit,
                                            sammyINPyw = elim_sammyINPyw , 
                                            options = elim_opts
                            )

In [ ]:
hist = elimi.eliminate(ladder_df= start_ladder)

In [ ]:
# true - using JEFF? just for comparison

true_chars = elimi.evaluate_prior(jeff_parameters) 
print(true_chars.chi2)

In [ ]:
# just to show how much we aliminated with the given threshold value.
print(f'Eliminated from {hist.ladder_IN.shape[0]} res -> {hist.ladder_OUT.shape[0]}')
print(f'Elim took {np.round(hist.elim_tot_time,2)} sec')

In [ ]:
# save history?

# can we save the history not to refit all the time?

fitted_elim_case_data = {
        'datasets' : datasets,
        'covariance_data' : covariance_data,
    'experiments': experiments,
    'true_chars': true_chars, # note, jeff are used as true here
    'Ta_pair': Ta_pair,
}

f_name_to_save = f'sf_{N_red}_er[{np.min(energy_range_all)}_{np.max(energy_range_all)}]_chi2allowed_{chi2_allowed}'

saved_hist = elim_addit_funcs.save_obj_as_pkl(folder_name=savefolder, file_name=f'hist_{f_name_to_save}.pkl', obj=hist)
saved_data = elim_addit_funcs.save_obj_as_pkl(folder_name=savefolder, file_name=f'dataset_{f_name_to_save}.pkl', obj=fitted_elim_case_data)


In [ ]:
# plot the final selected fit?
# images production from history

prior_level = max(hist.elimination_history.keys())
prior_numres = hist.elimination_history[prior_level]['input_ladder'].shape[0]
print(f'Initial ladder, num of res.: {prior_numres}')

min_level_passed_test = prior_level # level - key in the hist..
min_N_res_passed_test = prior_level - 1

levels = []
N_ress = []
chi2_s = []

for level in hist.elimination_history.keys():
        
    numres = hist.elimination_history[level]['selected_ladder_chars'].par_post.shape[0]
    pass_test = hist.elimination_history[level]['final_model_passed_test']

    #print(f'level {level}, # of resonances: {numres},  passed the test: {pass_test}')

    if (pass_test and level<min_level_passed_test):
        min_level_passed_test = level

        min_N_res_passed_test = numres
    
    levels.append(level)
    chi2_s.append(np.sum(hist.elimination_history[level]['selected_ladder_chars'].chi2_post))
    N_ress.append(numres)

# plotting    

# differences in chi2 values between 2 models
chi2_diffs = np.diff(chi2_s, prepend=chi2_s[0]) 

fig, (ax1, ax2) = subplots(2, 1, figsize = fig_size, sharex=True, gridspec_kw={'height_ratios': [2, 1]})

# First subplot for the original chi2 values
ax1.plot(N_ress, chi2_s, marker='o')
ax1.axvline(x=min_N_res_passed_test, color='r', linestyle='--')

ax1.set_ylabel('$\chi^2$')
ax1.grid(True)

# changes in chi2

ax2.plot(N_ress, chi2_diffs, marker='o', color='green')
ax2.axvline(x=min_N_res_passed_test, color='r', linestyle='--')
ax2.set_xlabel(r'$N_{res}$')
ax2.set_ylabel('Change in $\chi^2$')
ax2.invert_xaxis()
ax2.grid(True)

tight_layout()
f_name_to_save = f'hist_sf_{N_red}_er[{np.min(energy_range_all)}_{np.max(energy_range_all)}]_chi2allowed_{chi2_allowed}'
fig.savefig(savefolder+f'{f_name_to_save}.png')
show()

In [ ]:
# plotting data from history
def show_plot_from_hist(
        datasets,
        experiments,
        true_chars,
        true_pars,

        level_to_compare: int,
        min_level_passed_test: int,
        elim_hist: dict,
        addit_title_str: str = ''
        ):
    
    fits = elim_hist[level_to_compare]['selected_ladder_chars'].pw_post
    fits_chi2 = elim_hist[level_to_compare]['selected_ladder_chars'].chi2_post

    prior_fit = elim_hist[prior_level]['selected_ladder_chars'].pw
    priors_chi2 = elim_hist[prior_level]['selected_ladder_chars'].chi2

    # outfit
    fig = plot_datafits(datasets, experiments, 
        fits=fits, fits_chi2=fits_chi2, f_model_name=f'AF + el. {N_red}',
        priors = prior_fit, priors_chi2=priors_chi2, pr_model_name=f'AF, red. to {N_red}',
        true=true_chars.pw, t_model_name='JEFF (w/o autofit)',
        true_chi2 = true_chars.chi2,
        true_pars = true_pars,
        fit_pars = elim_hist[level_to_compare]['selected_ladder_chars'].par_post,
        prior_pars = elim_hist[prior_level]['input_ladder'],
        title = f'Fit, Prior & True comparison, model # {level_to_compare}, best selected: {min_level_passed_test} '+addit_title_str,
        show_spingroups=True
        )
    fig.tight_layout()
    return fig

In [ ]:
N_res_to_view = min_N_res_passed_test
level_to_compare = N_res_to_view+1

fig = show_plot_from_hist(datasets = datasets,
                          experiments=experiments,
                          true_chars = true_chars,
                          true_pars = true_chars.par,
                          level_to_compare=level_to_compare,
                          min_level_passed_test= min_level_passed_test,
                          elim_hist = hist.elimination_history,
                          addit_title_str=', $\Delta\chi^2$ = '+str(chi2_allowed)
                          )

fig.savefig(savefolder+f'elim_result_sel_sf_{N_red}_er[{np.min(energy_range_all)}_{np.max(energy_range_all)}]_chi2allowed_{chi2_allowed}.png')

In [ ]:
# table for analysis of the models - produce chi2

# sums - for all datasets
NLLW_s = []
chi2_s = []
aicc_s = []
bicc_s = []
N_res_s = []
test_pass = []

for level in hist.elimination_history.keys():
        
    numres = hist.elimination_history[level]['selected_ladder_chars'].par_post.shape[0]
    pass_test = hist.elimination_history[level]['final_model_passed_test']

    # print(f'level {level}, # of resonances: {numres},  passed the test: {pass_test}')
    cur_ch_dict = elim_addit_funcs.characterize_sol(Ta_pair=Ta_pair,
                     datasets=datasets,
                     experiments=experiments,
                     sol = hist.elimination_history[level]['selected_ladder_chars'],
                     covariance_data = covariance_data
                     )
    
    N_res_s.append(numres)
    test_pass.append(pass_test)
    chi2_s.append(sum(cur_ch_dict['chi2_stat']))
    
    # aicc_s.append(sum(cur_ch_dict['aicc']))
    # bicc_s.append(sum(cur_ch_dict['bicc']))

    aicc_s.append(cur_ch_dict['aicc_entire_ds'])
    bicc_s.append(cur_ch_dict['bic_entire_ds'])

    NLLW_s.append(sum(cur_ch_dict['NLLW'].values()))


    # levels.append(level)
    # chi2_s.append(np.sum(hist.elimination_history[level]['selected_ladder_chars'].chi2_post))
    # N_ress.append(numres)
    print(cur_ch_dict )

# combine to one DataFrame
table_df = pd.DataFrame.from_dict({
    'N_res': N_res_s,
    'passed': test_pass,
    'sum_chi2': chi2_s,
    'sum_NLLW': NLLW_s,
    'AICc': aicc_s,
    'BIC': bicc_s

})

# calculating deltas in BIC and AICc
table_df['delta_AICc_best'] = table_df['AICc'] - table_df['AICc'].min()
table_df['delta_BIC_best'] = table_df['BIC'] - table_df['BIC'].min()
table_df['delta_chi2_prev'] = table_df['sum_chi2'].diff().fillna(0)
table_df['delta_chi2_best'] = table_df['sum_chi2'] - table_df['sum_chi2'].min()

# saving comparison table
table_df.to_csv(path_or_buf=savefolder+f'comparison_sf_{N_red}_er[{np.min(energy_range_all)}_{np.max(energy_range_all)}]_chi2allowed_{chi2_allowed}.csv')
table_df

In [ ]:
# Print the DataFrame without the index
print(table_df.to_csv(index=False))